In [1]:
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
import tensorflow as tf
import numpy as np
sess = tf.Session()

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#todo 需要将每个图片，处理成28*28的相关的图片
resize_result = np.resize(mnist.train.images[0],new_shape=[28,28])

In [3]:
resize_result.shape

(28, 28)

In [4]:
# 尝试将对应的shape扩充一个维度
var1 = tf.Variable(initial_value=resize_result,dtype=tf.float32)
var2 = tf.expand_dims(input=var1,axis=2)
# 只输出了shape的长度
print("var1.shape:",tf.shape(var1))
print("var2.shape:",tf.shape(var2))
# 输出shape的具体的值
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(tf.shape(var1)))
print(sess.run(tf.shape(var2)))

var1.shape: Tensor("Shape:0", shape=(2,), dtype=int32)
var2.shape: Tensor("Shape_1:0", shape=(3,), dtype=int32)
[28 28]
[28 28  1]


In [5]:
# 定义输入的placeholder
x = tf.placeholder(shape=[None,28,28,1],dtype=tf.float32)
y_true =tf.placeholder(shape=[None,10],dtype=tf.float32)

In [6]:
# 定义stripe的shape，第一个和最后一个必须为1
stride_shape1=[1,2,2,1]
# 定义卷积神经网络
# 创建卷积核,设置5个卷积核
#conv_kernel = tf.Variable(dtype=shape=[3,3,1,5])
with tf.variable_scope("layer1",reuse=tf.AUTO_REUSE):
    conv_kernel1 = tf.get_variable(name="conv1",shape=[3,3,1,64],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer_conv2d())
    # 需要了解对应的参数
    conv1 = tf.nn.conv2d(input=x,filter=conv_kernel1,padding='SAME',strides=stride_shape1)
    bias1 = tf.Variable(tf.constant(0.1,shape=[64]))
    features1 = conv1+bias1
    # 计算产出的结果的大小
    #ceil[(28-3)/2]+1,第13个位置，1+（13-1）*2 = 25,结尾为25+stripe_step=25+2=27,还可以再添加一个
    #，所以conv的shape应该为[batch_size,13,13,5]
    # 定义激活层 
    act1 = tf.nn.relu(features=features1,name="act1")
    # 定义相关的max_pooling层,计算方式和conv层一样,ciel((14-2)/2)+1,max1的shape应该为[100,7,7,5]
    max1 = tf.nn.max_pool(value=act1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='max1')
    





In [7]:
type(tf.shape(max1))

tensorflow.python.framework.ops.Tensor

In [8]:
print(tf.shape(max1))
print(max1.shape)

Tensor("Shape_5:0", shape=(4,), dtype=int32)
(?, 7, 7, 64)


In [9]:
# 定义stripe的shape，第一个和最后一个必须为1
stride_shape2=[1,2,2,1]
# 定义卷积神经网络
# 创建卷积核,设置5个卷积核
#conv_kernel = tf.Variable(dtype=shape=[3,3,1,5])
with tf.variable_scope("layer2",reuse=tf.AUTO_REUSE):
    conv_kernel2 = tf.get_variable(name="conv2",shape=[3,3,max1.shape[-1],32],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer_conv2d())
    # 需要了解对应的参数
    conv2 = tf.nn.conv2d(input=max1,filter=conv_kernel2,padding='SAME',strides=stride_shape2)
    
    bias2 = tf.Variable(tf.constant(0.1,shape=[32]))
    features2 = conv2+bias2
    # todo 使用wx+b的形式，在激活层之前设置conv2
    # 计算产出的结果的大小
    #ceil[(7-3)/2]+1, 产出结果应该为[batch_size,4,4,3]
    # 定义激活层 
    act2 = tf.nn.relu(features=features2,name="act2")
    # 定义相关的max_pooling层,计算方式和conv层一样,ceil((4-2)/2)+1,max1的shape应该为[100,2,2,3]
    max2 = tf.nn.max_pool(value=act2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='max2')
    





In [10]:
# 定义全连接
# 参考网址:
with tf.variable_scope("fully_conn",reuse=tf.AUTO_REUSE):
    reshape_size=max2.shape[1].value*max2.shape[2].value*max2.shape[3].value
    #print("type reshape_sizee:",type(reshape_size),", value:",reshape_size)
    # 直接采用dense的方式
    #weights = tf.get_variable(shape=[None,reshape_size],)
    # 先将产出结果打平，进行相关的全连接
    flat = tf.reshape(tensor=max2,shape=[-1,reshape_size],name="flat")
    # 打平后，使用matmul计算全连接，并进行激活
    n_class=10
    fully1=tf.layers.dense(name="full1",units=50,inputs=flat,activation=tf.nn.relu,kernel_initializer=tf.truncated_normal_initializer,bias_initializer=tf.zeros_initializer,reuse=tf.AUTO_REUSE)
    
    # 定义使用soft
    
    fully2 = tf.layers.dense(name="full2",units=n_class,inputs=fully1,activation=tf.nn.softmax,kernel_initializer=tf.truncated_normal_initializer,bias_initializer=tf.zeros_initializer,reuse=tf.AUTO_REUSE)
    
    
    
    
    
    


In [11]:
print(x.shape)
print(fully2.shape)

(?, 28, 28, 1)
(?, 10)


In [12]:
# 定义相关的损失函数
with tf.variable_scope("loss",reuse=tf.AUTO_REUSE):
    # 使用交叉熵的方式，由于已经使用了softmax的方式，所以使用交叉熵
    # 交叉熵的计算公式,此处对交叉熵进行截断
    multi_result =-tf.multiply(y_true, tf.log(tf.clip_by_value(fully2,1e-10,1.1)))
    # 求和，获取每个样本的交叉熵
    ce = tf.reduce_sum(multi_result, reduction_indices=[1])
    # 计算整个batch中的交叉熵
    cost = tf.reduce_mean(ce)
    # 定义优化函数
    # opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = opt.minimize(cost)
    
    


In [13]:
def print_with_batch_size(sess,need_print_tensor):
    init = tf.global_variables_initializer()
    sess.run(init)
    batch_size = 100
    xs_batch,ys_batch = mnist.train.next_batch(batch_size=batch_size)
    # 对输入的数据，进行相应的reshape
    xs_batch_reshape = np.reshape(xs_batch,newshape=[batch_size,28,28,1])
    result = sess.run(need_print_tensor,feed_dict={x:xs_batch_reshape,y_true:ys_batch})
    print("shape result:",result.shape)
    print("value result:",result)
    

In [14]:
# 注意注意注意！！！！！！！！！，在执行train之前，需要注释掉所有的该代码，不然会影响loss的迭代和优化。
# print_with_batch_size(sess,fully1)

In [15]:
#print_with_batch_size(sess,fully2)

In [16]:
#检查最后输出的值是否在相应的范围内
#print_with_batch_size(sess,multi_result)

In [17]:
# 定义相关的损失函数和相关的优化函数

def do_train(sess):
    print_size = 200
    init = tf.global_variables_initializer()
    sess.run(init)
    count = 0 
    for i in range(5000):
        # 进行相关的训练
        batch_size = 100
        count = count +1 
        xs_batch,ys_batch = mnist.train.next_batch(batch_size=batch_size)
        # 对输入的数据，进行相应的reshape
        xs_batch_reshape = np.reshape(xs_batch,newshape=[batch_size,28,28,1])
        #print("xs_batch_reshape: ",np.shape(xs_batch_reshape),", dtype: ",xs_batch_reshape.dtype)
        _,loss = sess.run([train_op,cost],feed_dict={x:xs_batch_reshape,y_true:ys_batch})
        
        if count % print_size == 0:
            print("loss: ",loss)
        
            
    


In [18]:
do_train(sess)

loss:  0.41283026
loss:  0.1834982
loss:  0.42627418
loss:  0.30872172
loss:  0.16397482
loss:  0.19438943
loss:  0.26425335
loss:  0.25175896
loss:  0.30066878
loss:  0.16438584
loss:  0.1740794
loss:  0.083542675
loss:  0.09893694
loss:  0.10121113
loss:  0.05956073
loss:  0.0608853
loss:  0.10654433
loss:  0.070497215
loss:  0.31999287
loss:  0.073016904
loss:  0.04856941
loss:  0.17581703
loss:  0.1226931
loss:  0.18163356
loss:  0.1435606


In [19]:
def cal_accu(sess,predict_layer):
    #init = tf.global_variables_initializer()
    #sess.run(init)
    # 将输入数据预处理
    x_test= np.reshape(mnist.test.images,newshape=[mnist.test.images.shape[0],28,28,1])
    # 获取预测，最后的softmax层为full2
    predict_result = sess.run(predict_layer,feed_dict={x:x_test,y_true:mnist.test.labels})
    # 获取预测标签
    all_is_equal = tf.equal(tf.arg_max(predict_result,dimension=1),tf.arg_max(mnist.test.labels,dimension=1))
    #print_with_batch_size(sess,all_is_equal)
    # 计算精确率
    print("all_is_equal.dtype:",all_is_equal.dtype)
    accu = tf.reduce_mean(tf.cast(all_is_equal,tf.float32))
    #accu = tf.reduce_mean(all_is_equal)
    print("accu: ",sess.run(accu))
    
    

In [20]:
cal_accu(sess,fully2)

Instructions for updating:
Use `argmax` instead
all_is_equal.dtype: <dtype: 'bool'>
accu:  0.9617
